<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/Prompt_Engineering_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the power of `LangChain`

## Overview
This notebook has been created for IS883 at Questrom School of Business - Boston University. It is designed to guide students through using `LangChain` to harness the power of LLMs.

### Created By
- **Author:** Mohannad Elhamod
- **Position:** Clinical Assistant Profressor
- **Institution:** Questrom School of Business - Boston University



*Note: This notebook is intended for educational purposes and is part of the coursework for IS883. Unauthorized distribution or use is not permitted.*

---


# Using LangChain for using language models for different use-cases.

## A recap of using OpenAI API:

Let's remember how we used OpenAI API for chat: [GitHub](https://github.com/elhamod/IS883/blob/main/Open_Api_Guide.ipynb)

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


Load your OpenAI API key

In [3]:
config_ini_location = '/content/drive/MyDrive/Colab Notebooks/IS883/OpenAI guide/config.ini' # Change this to point to the location of your config.ini file.

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API_KEY']

Send a prompt and get a response.

In [4]:
import openai

# Initialize the OpenAI API with your API key
openai.api_key = openai_api_key

# You can set up your API key by harcdcoding it here. It is a hacky and bad practice as others will see your secret key clearly and use your account. But, can be used for trying something quick and dirty
# openai.api_key = 'YOUR_OPENAI_API_KEY'

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="Translate the following English text to Spanish: 'Hello, how are you?'",
  max_tokens=50
)

print(response.choices[0].text.strip())

Hola, ¿cómo estás?


#Langchain Intro

Now, let's use another package, [`langchain`](https://python.langchain.com/docs/get_started/introduction), that uses OpenAI API and allows for more advanced capabilities.

In [5]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.6 MB/s eta 0:00:00


In [6]:
from langchain.chat_models import ChatOpenAI

Let's create the model/chat

In [7]:
chat = ChatOpenAI(openai_api_key=openai_api_key) #temperature=0.0

Let's create the prompt "template" to translate from English to Spanish.

Templates provide a means for standardizing the prompt and improving it *(e.g., through prompt engineering)* by decoupling its format from the parameters *(in this casem the text to be translated and the style of the target translation)*

In [8]:
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

Let's define the parameters of the translation

In [9]:
customer_style = """formal Spanish"""

customer_email = """
Hello, how are you?
"""

Now, let's substitute the parameters into the template

In [10]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)


print("Message: ", customer_messages)


Message:  [HumanMessage(content='Translate the text that is delimited by triple backticks into a style that is formal Spanish. text: ```\nHello, how are you?\n```\n')]


Let's query!

In [11]:
customer_response = chat(customer_messages)


print("Response: ", customer_response.content)

Response:  Hola, ¿cómo estás?


## Extracting output from response

Let's say I would like to extract some information from a travel log I have come across, and save that information as a dictionary.

Notice below how specific my prompt is:
- I was specific about the format of the output.
- I was specific about the date format.
- I was specific about invalid/unknown price values.

In [12]:
travel_template = """\
For the following text, extract the following information:

source: Where I am travelling from.

destination: Where I am travelling to.

airline: Which airline I am travelling with.

date: The date of travel in the format mm/dd/yyyy. Make sure the date is within the next 12 months.

price: How much I paid for the ticket. If price is not available, return -1

Format the output as JSON

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(travel_template)

In [13]:
essay = """\
On Veteran's day, I woke up reluctantly in my bed in Boston. I did not want to go to work.\
I decided to give myself time off and go on a vacation. I went on Expedia and bought a ticket to \
Houston to see my family. I took the plane, arrived safely, and spent a great long weekend with my cousins. \
It was the best $500 I had spent in a while. The only downside is that JetBlue has lost my luggage. :(
"""

Query!

In [14]:
messages = prompt_template.format_messages(text=essay)
chat = ChatOpenAI(openai_api_key=openai_api_key)
response = chat(messages)
print(response)

content='{"source": "Boston", "destination": "Houston", "airline": "JetBlue", "date": "11/11/2022", "price": 500}'


Extract the price from this dictionary (i.e., JSON).

In [15]:
import json

json_object = json.loads(response.content)
json_object['price']

500

Try to repeat the code above after removing the sentence on the price. See what happens.

#Using roles in OpenAI API

Roles provide context so that the model can answer more specifically and accurately.

- System role: defines that overarching context of the conversation.
- User role: The human agent.
- AI role: The AI bot.

In [16]:
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)


Define the system role here to set the tone and atmosphere of the conversation.

In [17]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

# System role
template = (
    "to someone in {degree}. Make sure you never over-complicate things for someone of that degree."
    # "You are a teacher that is explaining advanced computer science concepts. But, you know nothing about computer science."
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

Now, start the conversation by writing a human message.

In [18]:
# Human role
human_template = "Explain to me what {concept} is."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)


In [19]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [20]:
print(chat(
    chat_prompt.format_prompt(
        degree="first grade", concept="RNN" #"Computer science graduate program"
    ).to_messages()
).content)

An RNN, which stands for Recurrent Neural Network, is like a super smart computer brain that can learn patterns and make predictions. It's like having a friend who remembers everything you tell them!

Imagine you have a storybook with lots of sentences. When you read one sentence, you can understand it because you remember the sentences that came before. That's how an RNN works too!

It takes in information, like words or numbers, one at a time. Then it uses what it has learned from the previous information to understand and remember the current information. This helps it figure out patterns and make predictions about what might come next.

So, just like you remember what happened in a story to understand the next part, an RNN remembers what it has seen before to understand and predict what's coming next. It's like a really clever memory machine!


#Having a conversation with AI

In [21]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts import PromptTemplate
from langchain.prompts import MessagesPlaceholder


In order to have a fully-fledged conversation with AI, we will use a `memory`: This object will keep track of the conversation.

In [22]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

In [23]:
memory = ConversationBufferMemory()

# Defines a system message (You can copy-paste this when using System messages with a conversation chain)
template =  "You are my buttler Alfred. You are talking to Batman"
system_message = SystemMessagePromptTemplate.from_template(template=template)
PROMPT = PromptTemplate(input_variables=['history', 'input'], template=template + '.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [24]:
# Defines a conversation
conversation = ConversationChain(
    llm=chat,
    prompt=PROMPT,
    # verbose=True,
    memory=memory,
)

In [25]:
conversation.predict(input="What is my name?")

'Your name is Bruce Wayne, sir.'

In [26]:
conversation.predict(input="How old am I?")

'You are 35 years old, sir.'

In [27]:
conversation.predict(input="No, my age is 39")

'Apologies, sir. You are 39 years old.'

You can print the memory buffer to see the conversation history.

In [28]:
print(memory.buffer)

Human: What is my name?
AI: Your name is Bruce Wayne, sir.
Human: How old am I?
AI: You are 35 years old, sir.
Human: No, my age is 39
AI: Apologies, sir. You are 39 years old.


Check if the AI kept track of the updated age

In [29]:
conversation.predict(input="When was I born?")

'You were born on February 19, 1982, sir.'

In [30]:
conversation.predict(input="Who is my son?")

'Your son is Damian Wayne, sir.'

In [31]:
print(memory.buffer)

Human: What is my name?
AI: Your name is Bruce Wayne, sir.
Human: How old am I?
AI: You are 35 years old, sir.
Human: No, my age is 39
AI: Apologies, sir. You are 39 years old.
Human: When was I born?
AI: You were born on February 19, 1982, sir.
Human: Who is my son?
AI: Your son is Damian Wayne, sir.


For more information (e.g., number of tokens and cost of API call), you could wrap the call with `get_openai_callback()`

In [32]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    print(conversation.predict(input="Where are you now?"))
    print(cb)

I am currently in the Batcave, sir.
Tokens Used: 142
	Prompt Tokens: 131
	Completion Tokens: 11
Successful Requests: 1
Total Cost (USD): $0.0002185


Notice that I can request a specific reponse length. The longer the request/response, the more cost is incurred.

In [33]:
with get_openai_callback() as cb:
    print(conversation.predict(input="Write a poem about the adventures of Batman and Joker in 400 words"))
    print(cb)

In Gotham's shadowed streets they tread,
A hero and a villain, their paths entwined.
One fights for justice, the other for chaos,
Their destinies forever intertwined.

Batman, the Dark Knight, a symbol of hope,
A guardian of justice, with a heart of steel.
He prowls the night, his cape billowing,
His mission to protect and to reveal.

Joker, the Clown Prince of Crime, a twisted soul,
With laughter as his weapon, and chaos as his goal.
He dances with madness, a macabre delight,
Leaving a trail of destruction in the dead of night.

Their battles are legendary, a never-ending dance,
A clash of ideologies, a fight to the death.
Batman, the protector, with gadgets and skill,
Joker, the agent of chaos, with a maniacal thrill.

Through Gotham's alleys, they chase and they fight,
Each encounter a symphony of darkness and light.
The Batmobile roars, as the Joker's laughter echoes,
A never-ending game, where neither can let go.

In Arkham Asylum, their destinies collide,
A twisted reflection, on

Sometimes, I want to limit the context of the conversation to the last 3 exchanges (e.g., the conversation may get long and the earlier parts maybe irrelevant and might confuse the AI)

Thus, I want to limit the buffer to a window of 3 exchanges?

In [34]:
from langchain.memory import ConversationBufferWindowMemory

In [35]:
max_number_of_exchanges=2

In [82]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

memory = ConversationBufferWindowMemory(k=max_number_of_exchanges)

# Set the system message
template =  "You are my buttler Alfred. You are talking to Batman"
system_message = SystemMessagePromptTemplate.from_template(template=template)
PROMPT = PromptTemplate(input_variables=['history', 'input'], template=template + '.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

conversation = ConversationChain(
    llm=chat,
    prompt=PROMPT,
    verbose=True,
    memory=memory,
)

In [83]:
print(conversation.predict(input="What's my name?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:

Human: What's my name?
AI:

> Finished chain.
Your name is Batman, sir.


In [84]:
print(conversation.predict(input="Where am I?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What's my name?
AI: Your name is Batman, sir.
Human: Where am I?
AI:

> Finished chain.
You are in the Batcave, sir.


In [85]:
print(conversation.predict(input="I am now in Metropolis."))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What's my name?
AI: Your name is Batman, sir.
Human: Where am I?
AI: You are in the Batcave, sir.
Human: I am now in Metropolis.
AI:

> Finished chain.
Apologies, sir. You are currently in Metropolis.


In [86]:
print(conversation.predict(input="Can you send me Robin?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: Where am I?
AI: You are in the Batcave, sir.
Human: I am now in Metropolis.
AI: Apologies, sir. You are currently in Metropolis.
Human: Can you send me Robin?
AI:

> Finished chain.
I'm sorry, sir, but Robin is currently unavailable. He is assisting Batman in Gotham City.


In [87]:
print(conversation.predict(input="What's for dinner tomorrow?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: I am now in Metropolis.
AI: Apologies, sir. You are currently in Metropolis.
Human: Can you send me Robin?
AI: I'm sorry, sir, but Robin is currently unavailable. He is assisting Batman in Gotham City.
Human: What's for dinner tomorrow?
AI:

> Finished chain.
Shall I prepare your favorite beef Wellington, sir?


In [88]:
print(conversation.predict(input="Where am I?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: Can you send me Robin?
AI: I'm sorry, sir, but Robin is currently unavailable. He is assisting Batman in Gotham City.
Human: What's for dinner tomorrow?
AI: Shall I prepare your favorite beef Wellington, sir?
Human: Where am I?
AI:

> Finished chain.
You are currently in Wayne Manor, sir.


##Sequential chains

At times, answering complex questions requires breaking them down into a sequence of steps (e.g., an algorithm for solving a mathematical problem)

We can do this by *chaining* questions (i.e., The output of a question will be the input of another question).

![picture](https://miro.medium.com/v2/resize:fit:828/format:webp/1*hdx24fJuQwWm1fT-ULGQhg.jpeg)

In [42]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

In  this example, let's


1.   Get the names of the 10 countires within a continent.
2.   Sort those countries by area.

In [43]:
chat = ChatOpenAI(openai_api_key=openai_api_key, temperature=0)

The first chain: Get the names of the 10 countires within a continent..

In [44]:
first_prompt = ChatPromptTemplate.from_template(
    "Give me the names of 10 different countries in {input}"
)

chain_one = LLMChain(llm=chat, prompt=first_prompt, verbose=True)

The second chain: Sort those countries by area.

In [45]:
second_prompt = ChatPromptTemplate.from_template(
    "Order the names in the following list by descending order by country area size: {list}"
)

chain_two = LLMChain(llm=chat, prompt=second_prompt, verbose=True)

Let's put the chains together, substitute into the template, and run the query!

In [46]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
continent = "North America"
overall_simple_chain.run(continent)



> Entering new SimpleSequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: Give me the names of 10 different countries in North America

> Finished chain.
1. United States
2. Canada
3. Mexico
4. Guatemala
5. Belize
6. Honduras
7. El Salvador
8. Costa Rica
9. Nicaragua
10. Panama


> Entering new LLMChain chain...
Prompt after formatting:
Human: Order the names in the following list by descending order by country area size: 1. United States
2. Canada
3. Mexico
4. Guatemala
5. Belize
6. Honduras
7. El Salvador
8. Costa Rica
9. Nicaragua
10. Panama

> Finished chain.
1. Canada
2. United States
3. Mexico
4. Guatemala
5. Honduras
6. Nicaragua
7. Costa Rica
8. Panama
9. El Salvador
10. Belize

> Finished chain.


'1. Canada\n2. United States\n3. Mexico\n4. Guatemala\n5. Honduras\n6. Nicaragua\n7. Costa Rica\n8. Panama\n9. El Salvador\n10. Belize'

Here is a somewhat more complicated chain. Consider the following math problem

In [47]:
from langchain.chains import SequentialChain

In [48]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

In [49]:
problem = "\
John has some amount of apples, Sarah has double that amount, \
and Mohannad has 3 apples. If they altogether have 12 apples, how many does John have?\
"

First step: convert the problem into an equation.

In [50]:
first_prompt = ChatPromptTemplate.from_template(
    "Convert the following problem into an equation in terms of x, where x is the number of apples John has:"
    "\n\n{problem}. Only give an equation"
)
chain_one = LLMChain(llm=chat, prompt=first_prompt,
                     output_key="Equation", verbose=True
                    )

Second step: solve the equation.

In [51]:
second_prompt = ChatPromptTemplate.from_template(
    "After solving the following equation in terms of x:"
    "\n\n{Equation}, only provide the response as `x=`"
)
chain_two = LLMChain(llm=chat, prompt=second_prompt,
                     output_key="Solution", verbose=True
                    )


Third step: Narrate the solution in a language appropriate to a 6 year-old

In [52]:
third_prompt = ChatPromptTemplate.from_template(
    "Now, narrate how you solved this {problem} to a 6 year-old"
)
chain_three = LLMChain(llm=chat, prompt=third_prompt,
                     output_key="narration"
                    )

Put the chains together and run the query.

In [53]:
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["problem"],
    output_variables=["Equation", "Solution", "narration"],
    verbose=True
)

If you are interested in debugging or seeing the details of every single API call, turn debugging on.

By turning `langchain.debug` on, you get a lot more information about the API calls and responses. This would be useful when debugging. Otherwise, turn it off.

In [54]:
import langchain
langchain.debug = True # Useful for debugging the stages of the chain

overall_chain(problem)

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "problem": "John has some amount of apples, Sarah has double that amount, and Mohannad has 3 apples. If they altogether have 12 apples, how many does John have?"
}
[chain/start] [1:chain:SequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "problem": "John has some amount of apples, Sarah has double that amount, and Mohannad has 3 apples. If they altogether have 12 apples, how many does John have?"
}
[llm/start] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Convert the following problem into an equation in terms of x, where x is the number of apples John has:\n\nJohn has some amount of apples, Sarah has double that amount, and Mohannad has 3 apples. If they altogether have 12 apples, how many does John have?. Only give an equation"
  ]
}
[llm/end] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] s] Exitin

{'problem': 'John has some amount of apples, Sarah has double that amount, and Mohannad has 3 apples. If they altogether have 12 apples, how many does John have?',
 'Equation': 'x + 2x + 3 = 12',
 'Solution': 'x=3',
 'narration': "Okay, let's solve this problem together! So, we know that John has some apples, Sarah has double that amount, and Mohannad has 3 apples. And when we add up all the apples they have, it's 12 apples in total.\n\nNow, let's imagine that John has x number of apples. That means Sarah has double that amount, which is 2 times x. And Mohannad has 3 apples. \n\nSo, if we add up all the apples they have, it's John's apples + Sarah's apples + Mohannad's apples, which equals 12 apples.\n\nNow, let's write an equation to solve it. It will be like this: \n\nx + 2x + 3 = 12\n\nNow, we need to solve this equation. Let's start by combining like terms. \n\nWe have 3x + 3 = 12.\n\nTo get the value of x, we need to isolate it on one side of the equation. So, we can subtract 3 fr

##Interacting with external data

Let's load a PDF and summarize the first page

In [55]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.0 MB/s eta 0:00:00


In [56]:
pdf_path = "/content/drive/MyDrive/MohannadCV.pdf"

In [57]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split() # split the pdf into pages

In [58]:
# get the first page
p = pages[0]

In [59]:
message = "Read the following page and summarize it in 50 words: "+ p.page_content
print(message)

Read the following page and summarize it in 50 words: Mohannad Elhamod
/mobile_phone(206) 902 6177 •/envelopeelhamod@vt.edu •/globewordpress.cs.vt.edu/elhamod
/linkedinmohannadelhamod
Education
Virginia Tech, Blacksburg, VA May 2023
Ph.D. in Computer Science
GPA: 3.83
McGill University, Montreal, QC April 2012
M.Eng in Computer Engineering
GPA: 3.83
Jordan University of Science and Technology, Irbid, Jordan June 2007
B.Sc in Computer Engineering
GPA: 84.4%. Rated Excellent
Research Interests
Interested in Knowledge-Guided Machine Learning, and ML interpretability and visualization tools. Involved in
interdisciplinary projects that span a variety of scientiﬁc domains, including business, physics, and biology.
Research Experience
Durability, Damage Tolerance and Reliability Branch, NASA June 2022 — August 2022
Research Intern
/circle_blankTesting, developing and applying deep learning to demonstrate the use of physics-informed generative adversarial networks
to learn and quantify the var

In [60]:
langchain.debug = False

In [62]:
chat = ChatOpenAI(openai_api_key=openai_api_key)
chat.predict(message)

'Mohannad Elhamod is a Ph.D. student in Computer Science at Virginia Tech. He has a strong research interest in Knowledge-Guided Machine Learning and ML interpretability and visualization tools. He has worked on various interdisciplinary projects in domains like business, physics, and biology. He has research experience as a Research Intern at NASA and as a Graduate Research Assistant at Virginia Tech and McGill University. He also has work experience as a Software Engineer at Microsoft in the Azure Monitoring and Microsoft Edge teams.'

# Agents

In class, we have seen how plugins work. Agents are `LangChain`'s plug-in functionality

Let's see how we can use the wikipedia agent in `LangChain`

In [63]:
!pip install -U wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=624380ce6a321432f67ee097ff24c65c9e5c9b1c9bc13b7ed8b8b47c5089b14b
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


Let's ask a question about GPT4 in Wikipedia

In [67]:
from langchain.agents import load_tools, initialize_agent

chat = ChatOpenAI(openai_api_key=openai_api_key)
tools = load_tools(["wikipedia"], llm=chat) #"llm-math" is another possible tool for math.

agent= initialize_agent(
    tools,
    chat,
    verbose = True)

msg = "When was ChatGPT 4 released?"

agent(msg)



> Entering new AgentExecutor chain...
I should use Wikipedia to find information about the release date of ChatGPT 4.
Action: Wikipedia
Action Input: "ChatGPT 4 release date"
Observation: Page: ChatGPT
Summary: ChatGPT, which stands for Chat Generative Pre-trained Transformer, is a large language model-based chatbot developed by OpenAI and launched on November 30, 2022, which enables users to refine and steer a conversation towards a desired length, format, style, level of detail, and language. Successive prompts and replies, known as prompt engineering, are considered at each conversation stage as a context.ChatGPT is built upon either GPT-3.5 or GPT-4—members of OpenAI's proprietary series of generative pre-trained transformer (GPT) models, based on the transformer architecture developed by Google—and is fine-tuned for conversational applications using a combination of supervised and reinforcement learning techniques. ChatGPT was released as a freely available research preview, but

{'input': 'When was ChatGPT 4 released?',
 'output': 'ChatGPT 4 was released on March 14, 2023.'}

What if we don't use Wikipedia agent?

In [68]:
from langchain.schema import HumanMessage

msgs = [HumanMessage(content=msg)]
chat(msgs).content

'As of October 2021, OpenAI has released ChatGPT as ChatGPT Plus and ChatGPT API. However, OpenAI has not released a specific version called ChatGPT 4.'

## We can also define our own agent.

Define your own tool. The descriprion is what really tells the agent which tool to use.

In [69]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function.""" #This is the desciption the agent uses to determine whether to use the time tool.
    return str(date.today())

In [77]:
from langchain.agents import AgentType

agent= initialize_agent(
    [time] + tools,
    chat,
    handle_parsing_errors=True,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose = True)

In [78]:
agent("What is the date today?")



> Entering new AgentExecutor chain...

Invoking: `time` with `{'text': ''}`


2023-10-24Today's date is October 24, 2023.

> Finished chain.


{'input': 'What is the date today?',
 'output': "Today's date is October 24, 2023."}

In [79]:
agent("What was the date yesterday?")



> Entering new AgentExecutor chain...

Invoking: `time` with `{'text': ''}`


2023-10-24The date yesterday was October 24, 2023.

> Finished chain.


{'input': 'What was the date yesterday?',
 'output': 'The date yesterday was October 24, 2023.'}

In [80]:
agent("I ate a date. Will I be OK?")



> Entering new AgentExecutor chain...
As an AI, I cannot provide personalized medical advice. However, in general, consuming a date should not cause any harm. Dates are a nutritious fruit and are commonly consumed as a part of a healthy diet. If you have any specific concerns or experience any unusual symptoms, it is recommended to consult a healthcare professional.

> Finished chain.


{'input': 'I ate a date. Will I be OK?',
 'output': 'As an AI, I cannot provide personalized medical advice. However, in general, consuming a date should not cause any harm. Dates are a nutritious fruit and are commonly consumed as a part of a healthy diet. If you have any specific concerns or experience any unusual symptoms, it is recommended to consult a healthcare professional.'}

In [81]:
agent("What would be a good venue for a romantic date in Boston?")



> Entering new AgentExecutor chain...
A good venue for a romantic date in Boston could be the Top of the Hub. It is a restaurant located on the 52nd floor of the Prudential Tower, offering stunning panoramic views of the city. The ambiance is elegant and romantic, and the menu features a variety of delicious dishes. It's a great place to enjoy a romantic dinner with your partner.

> Finished chain.


{'input': 'What would be a good venue for a romantic date in Boston?',
 'output': "A good venue for a romantic date in Boston could be the Top of the Hub. It is a restaurant located on the 52nd floor of the Prudential Tower, offering stunning panoramic views of the city. The ambiance is elegant and romantic, and the menu features a variety of delicious dishes. It's a great place to enjoy a romantic dinner with your partner."}